<a href="https://colab.research.google.com/github/shaddy2911/Generative-AI/blob/main/Generated_images_MRI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, Flatten, Dropout, BatchNormalization, LeakyReLU, Conv2D, Conv2DTranspose
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2


# Define paths to each class
data_path = '/content/drive/MyDrive/My Projects/brain_tumor_dataset/yes'

def load_images_from_folder(folder, target_size=(64, 64)):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, target_size)
            images.append(img)
    return np.array(images)

# Load images for the selected class
X_train = load_images_from_folder(data_path)

# Normalize to [0, 1] range
X_train = X_train / 255.0

# Expand dimensions to add channel
X_train = np.expand_dims(X_train, axis=-1)

# Define the input shape
img_shape = X_train.shape[1:]
latent_dim = 100

def build_generator():
    model = Sequential()

    model.add(Dense(8*8*256, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((8, 8, 256)))
    model.add(BatchNormalization(momentum=0.8))

    model.add(Conv2DTranspose(128, kernel_size=4, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))

    model.add(Conv2DTranspose(64, kernel_size=4, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))

    model.add(Conv2DTranspose(1, kernel_size=4, strides=2, padding='same', activation='tanh'))

    return model

generator = build_generator()

def build_discriminator():
    model = Sequential()

    model.add(Conv2D(64, kernel_size=4, strides=2, padding='same', input_shape=img_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, kernel_size=4, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Conv2D(256, kernel_size=4, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    return model

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

def build_gan(generator, discriminator):
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

discriminator.trainable = False
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

def train(X_train, class_name, epochs=10000, batch_size=32, save_interval=1000):
    half_batch = int(batch_size / 2)

    for epoch in range(epochs):
        # Train Discriminator
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        imgs = X_train[idx]

        noise = np.random.normal(0, 1, (half_batch, latent_dim))
        gen_imgs = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train Generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        valid_y = np.array([1] * batch_size)

        g_loss = gan.train_on_batch(noise, valid_y)

        # Print the progress
        print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {100*d_loss[1]}] [G loss: {g_loss}]")

        # If at save interval, save generated image samples
        if epoch % save_interval == 0:
            save_imgs(epoch, class_name)

def save_imgs(epoch, class_name):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)

    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1
    if not os.path.exists(f'images/{class_name}'):
        os.makedirs(f'images/{class_name}')
    fig.savefig(f"images/{class_name}/{class_name}_{epoch}.png")
    plt.close()

# Train the GAN for the selected class
class_name = 'yes'  # Change this to 'no' when training for the 'no' class
train(X_train, class_name, epochs=10000, batch_size=32, save_interval=1000)


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 24ms/step
7501 [D loss: 0.23681052401661873 | D accuracy: 90.625] [G loss: 4.7206549644470215]
1/1 [==============================] - 0s 19ms/step
7502 [D loss: 0.2798224985599518 | D accuracy: 93.75] [G loss: 3.6771035194396973]
1/1 [==============================] - 0s 25ms/step
7503 [D loss: 0.22204123437404633 | D accuracy: 87.5] [G loss: 4.580594539642334]
1/1 [==============================] - 0s 25ms/step
7504 [D loss: 0.1129351258277893 | D accuracy: 100.0] [G loss: 3.926546573638916]
1/1 [==============================] - 0s 19ms/step
7505 [D loss: 0.2967734783887863 | D accuracy: 90.625] [G loss: 2.52278733253479]
1/1 [==============================] - 0s 18ms/step
7506 [D loss: 0.18067973107099533 | D accuracy: 93.75] [G loss: 4.362698554992676]
1/1 [==============================] - 0s 25ms/step
7507 [D loss: 0.06217106990516186 | D accuracy: 100.0] [G loss: 4.925477027893066]
1/1 [